# Whatsapp - Chatbot

Controlar o Whatsapp Web via Python + Selenium + Firefox.

## Setup

In [44]:
import os, sys, platform
sys.path.append(os.path.abspath('..'))
if platform.system() == 'Windows':
    os.environ['COMSPEC'] = 'powershell'
FILE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', 'files'))

## Banco de dados

ORM para persistencia de contatos e mensagens do Whatsapp.

    PonyORM + SQLite

### Criando database

In [54]:
from pony import orm
db = orm.Database()

### Entidades

In [55]:
class Contato(db.Entity):
    id = orm.PrimaryKey(int, auto=True)
    nome = orm.Required(str)
    telefone = orm.Optional(str)
    ordem = orm.Required(int)


### Criando banco de dados e mapeando entidades

In [56]:
db.bind(provider='sqlite', filename=os.path.join(FILE_DIR, 'whatsapp.sqlite'), create_db=True)
db.generate_mapping(create_tables=True)

## Whatsapp WebDriver

In [57]:
import base64
import re
import time
from typing import Dict, List, Union, Optional


class WhatsAppWebDriver:
    
    def __init__(self, webdriver, contato: Contato):
        self.webdriver = webdriver
        self._contato = contato
    
    def acessar_site(self):
        whatsapp_url = 'https://web.whatsapp.com'
        if whatsapp_url not in self.webdriver.current_url:
            self.webdriver.get(whatsapp_url)

    def precisa_sincronizar(self) -> bool:
        if 'Para usar o WhatsApp no seu computador' in self.webdriver.page_source:
            return True
        else:
            return False
        
    def capturar_qrcode(self):
        self.recarregar_qrcode()
        canvas = self.webdriver.find_element_by_tag_name('canvas')
        canvas_base64 = self.webdriver.execute_script("return arguments[0].toDataURL('image/png').substring(21);", canvas)
        return f'data:image/png;base64{canvas_base64}"'

    def recarregar_qrcode(self):
        self.webdriver.find_element_by_class_name('_2nIZM').click()

    def carregar_listar_contatos(self) -> bool:
        self.__capturar_contatos()
        return True
    
    @orm.db_session
    def listar_contatos(self) -> List[str]:
        lista_contatos = self._contato.select().order_by(self._contato.ordem)
        return [contato.nome for contato in lista_contatos]
    
    def listar_contatos_e_telefones(self) -> List[Dict[str, str]]:
        lista_dados = []
        lista_contatos = self.listar_contatos()
        for nome in lista_contatos:
            self.__comecar()
            dado = self.contato_e_telefone(nome)
            lista_dados.append(dado)
        return lista_dados
    
    def contato_e_telefone(self, nome: str) -> Dict[str, str]:
        try:
            dado = {}
            dado['nome'] = nome
            self.clicar_contato(nome)
            elemento = self.webdriver.find_elements_by_xpath('//div[@class="PVMjB"]').pop()
            elemento.click()
            elemento = self.webdriver.find_elements_by_xpath('//*[@title="Dados do contato"]').pop()
            elemento.click()
            elemento = self.webdriver.find_elements_by_xpath(f'//span[@class="_1X4JR"]').pop(3)
            dado['telefone'] = elemento.text
            self.webdriver.find_element_by_xpath(f'//button[@class="t4a8o"]')
        except:
            ...    
        return dado
    
    def clicar_contato(self, nome: str):
        elemento = self.webdriver.find_element_by_xpath(f'//span[@dir and @title="{nome}"]')
        if elemento:
            elemento.click()
        return elemento

    def enviar_mensagem(self, nome: str, mensagem: str):
        contatos = list(filter(lambda c: c.get('nome') == nome, self._lista_contatos))
        if len(contatos) != 1:
            raise Exception('Não foi identificado o nome do contato!')
        self.__rolar_listar_contatos(contatos.pop().get('ordem'))
        mensagem = mensagem.strip()
        if not mensagem:
            return
        self.clicar_contato(nome)
        grupo_1 = self.webdriver.find_element_by_xpath('//footer[@class="_2vJ01"]')
        grupo_2 = grupo_1.find_element_by_xpath('//div[@class="_3uMse"]')
        grupo_2.click()
        grupo_3 = grupo_2.find_element_by_class_name('_3FRCZ')
        for caracter in mensagem:
            grupo_3.send_keys(caracter)
        botao_enviar = grupo_1.find_element_by_xpath('//div[@class="_1JNuk"]/button')
        botao_enviar.click()
        
    def __comecar(self):
        self.acessar_site()
        try:
            self.webdriver.find_element_by_class_name('_1MXsz').pop().click()
        except:
            pass
        if self.precisa_sincronizar():
            raise Exception('Precisar sincronizar o QRCode!')
#         self.webdriver.execute_script('document.getElementById("pane-side").scrollTo({top: 0, left: 0, behavior: "smooth"});')
#         time.sleep(3)
        self.webdriver.execute_script('document.getElementById("pane-side").scrollTo({top: 0, left: 0});')
    
    @orm.db_session
    def __capturar_contatos(self) -> List[Dict[str, Union[str, int]]]:
        global lista_contatos_externa
        lista_contatos_temp = {}
        
        def capturar_contatos(webdriver):
            lista_elementos = webdriver.find_elements_by_class_name('_210SC')
            for elemento in lista_elementos:
                contato = {}
                for ec in elemento.find_elements_by_class_name('_3CneP'):
                    contato['nome'] = ec.get_attribute('innerText')
                    break
                if contato.get('nome'):
                    transform = elemento.value_of_css_property('transform')
                    ordem = int(re.sub(r'^.+,\s(\d+)\)$', r'\1', transform))
                    contato['ordem'] = ordem
                    lista_contatos_temp[ordem] = contato

        self.__rolar_listar_contatos()
        capturar_contatos(self.webdriver)
            
        lista_keys = sorted(lista_contatos_temp.keys())
        ultimo_contato_anterior = {}
        ultimo_contato = lista_contatos_temp[lista_keys.pop()]

        while True:
            if ultimo_contato_anterior.get('ordem') == ultimo_contato.get('ordem'):
                break
            self.__rolar_listar_contatos(ultimo_contato.get('ordem') + 1)
            capturar_contatos(self.webdriver)
            lista_keys = sorted(lista_contatos_temp.keys())
            ultimo_contato_anterior = ultimo_contato
            ultimo_contato = lista_contatos_temp[lista_keys.pop()]
    
        [self._contato(nome=lista_contatos_temp[key]['nome'],
                 telefone='',
                 ordem=lista_contatos_temp[key]['ordem'])
                 for key in sorted(lista_contatos_temp.keys())]

    def __rolar_listar_contatos(self, ordem: Optional[int] = None):
        ordem = ordem if ordem else 0
        webdriver.execute_script("""
        document.getElementById('pane-side').scrollTo({
          top: #top#,
          left: 0,
          behavior: 'smooth'
        });
        """.replace('#top#', str(ordem)))
        time.sleep(1)


## Desafios

Executar quadro abaixo 1 vez para instanciar o navegador apenas 1 vez e não perder a instancia dele. Vai evitar de perder a sincronização do Whatsapp e o smartphone.

In [45]:
from modules.webdriver.firefox import WebDriverFirefox
webdriver = WebDriverFirefox(False, 1).webdriver

### Acessar o site do Whatsapp Web

Apenas acessar o site.

In [47]:
whatsapp = WhatsAppWebDriver(webdriver, Contato)
whatsapp.acessar_site()

### Capturar QRCode de sincronia com Whatsapp

Extrair o QRCode de sincronia do navegador com o smartphone e apresentar a imagem.

In [74]:
whatsapp = WhatsAppWebDriver(webdriver, Contato)
precisa_sincronizar = whatsapp.precisa_sincronizar()

if precisa_sincronizar:
    whatsapp.recarregar_qrcode()
    imagem_base64 = whatsapp.capturar_qrcode()

    # Gerando HTML com o QRCODE para validar
    html_qrcode = f'<html><body><img src="{imagem_base64}"></body></html>'
    with open(f'{FILE_DIR}/html_qrcode.html', 'w') as f:
        f.write(html_qrcode)
    os.system(f'{FILE_DIR}/html_qrcode.html')
    print('É necessario sincronizar!')

else:
    print('Não é necessario sincronizar!')
    

Não é necessario sincronizar!


### Listar todos os contatos

Listar todos os contatos do usuario.

In [58]:
whatsapp = WhatsAppWebDriver(webdriver, Contato)
whatsapp.carregar_listar_contatos()
whatsapp.listar_contatos()

['Anne Carolinne',
 'Marcelo (Apê11)',
 'Bruno (GFT)',
 'Rafael Bombi',
 'Apê11',
 'Cesar Augusto Euzebio',
 'Salete Panhota',
 'Meetup Tec',
 'Barriguinha mole',
 'Silmara Panhota',
 'Rita Silva',
 'Luiz Anselmo da Silva',
 'Rosemeire',
 'Geraldo Lima',
 'Nerdices (política)',
 'Cristina (Apê 11)',
 'Gislaine Coelho',
 'Luana (Apê11)',
 'Valdir Miranda',
 'Gislaine Coelho',
 'Alessandra Cassia Da Silva',
 'Gislaine Coelho',
 'Vinícius (Barbearia)',
 'Marcia (Allegrini)',
 '+55 11 96922-3615',
 'Cecilia Garcia Miranda',
 'Leo (Apê11)',
 'happy',
 'Victor Panhota Da Silva',
 'Vera Lucia Garcia',
 'Sueli (Assistente Da Cleo Dentista)',
 'Celia Panhota',
 'Bruno (imóvel. aí)',
 'Uma GFT menos Jovem',
 'Pedro (Dentista)',
 '+55 11 4116-3753',
 'Tadashi',
 'Jovenildo',
 'Rafael Manzoni',
 'Danilo De Luna Martins',
 'Andrios (GFT)',
 'Papete de hugo',
 'Irmandade Moto',
 'Julio (Gerente Personalité)',
 'Marina Marques Garcia de Miranda',
 'Simone Panhota',
 'Wellington Oliveira',
 'Pedro (Ra

In [59]:
Contato.select().show()

id |nome                               |telefone|ordem
---+-----------------------------------+--------+-----
1  |Anne Carolinne                     |        |0    
2  |Marcelo (Apê11)                    |        |72   
3  |Bruno (GFT)                        |        |144  
4  |Rafael Bombi                       |        |216  
5  |Apê11                              |        |288  
6  |Cesar Augusto Euzebio              |        |360  
7  |Salete Panhota                     |        |432  
8  |Meetup Tec                         |        |504  
9  |Barriguinha mole                   |        |576  
10 |Silmara Panhota                    |        |648  
11 |Rita Silva                         |        |720  
12 |Luiz Anselmo da Silva              |        |792  
13 |Rosemeire                          |        |864  
14 |Geraldo Lima                       |        |936  
15 |Nerdices (política)                |        |1008 
16 |Cristina (Apê 11)                  |        |1080 
17 |Gislai

### Enviar mensagem para um contato especifico

In [266]:
whatsapp = WhatsAppWebDriver(webdriver, Contato)
whatsapp.enviar_mensagem('Anne Carolinne', 'oi')

### Mensagens em massa

In [56]:
lista_contatos = [
 'Anne Carolinne',
 'Rosemeire',
 'Silmara Panhota',
 'Rita Silva',
 'Luiz Anselmo da Silva'
]

whatsapp = WhatsAppWeb(webdriver)

for contato in lista_contatos:
    whatsapp.enviar_mensagem(contato, 'Estou testando umas paradas no meu computador!')


### Obter dados do contato

In [134]:
whatsapp = WhatsAppWebDriver(webdriver, Contato)
# whatsapp.listar_contatos()
# whatsapp.enviar_mensagem('Anne Carolinne', 'Estou testando umas paradas no meu computador!')
# whatsapp.contato_e_telefone('Rita Silva')
print(repr(whatsapp.listar_contatos_e_telefones()))

[{'nome': '+55 11 96922-3615', 'telefone': '+55 11 96922-3615'}, {'nome': 'Agendamentos só chamar!'}, {'nome': 'Alessandra Cassia Da Silva', 'telefone': '+55 11 99265-8248'}, {'nome': 'Barriguinha mole'}, {'nome': 'Cecilia Garcia Miranda'}, {'nome': 'Cristina (Apê 11)', 'telefone': '+55 11 98199-2975'}, {'nome': 'Geraldo Lima', 'telefone': '+55 11 98163-1335'}, {'nome': 'Gislaine Coelho', 'telefone': '+55 11 96638-1918'}, {'nome': 'Leo (Apê11)', 'telefone': '+55 11 97504-2027'}, {'nome': 'Luana (Apê11)', 'telefone': '+55 11 98776-7177'}, {'nome': 'Marcia (Allegrini)'}, {'nome': 'Meetup Tec'}, {'nome': 'Nerdices (política)'}, {'nome': 'Salete Panhota'}, {'nome': 'Valdir Miranda', 'telefone': '+55 11 97395-0172'}, {'nome': 'Vinícius (Barbearia)', 'telefone': 'Agendamentos só chamar!'}]


In [82]:
elemento = webdriver.find_elements_by_xpath('//div[@class="PVMjB"]').pop()
elemento.attribute('innerHtml')
# elemento.click()
# elemento = webdriver.find_elements_by_xpath('//*[@title="Dados do contato"]').pop()
# elemento.click()
# elemento = webdriver.find_elements_by_xpath(f'//span[@class="_1X4JR"]').pop(3)


AttributeError: 'FirefoxWebElement' object has no attribute 'attribute'